In [1]:
# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os
import subprocess

subprocess.run(["pip", "install", "Werkzeug"])
subprocess.run(["pip", "install", "tensorflow"])

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/cnn"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/cnn"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
from sagemaker.tensorflow import TensorFlow

In [3]:
import argparse
import os
from sklearn.linear_model import LinearRegression
import pickle
import numpy as np


# CODE HELPER 1 - notice how data is saved -> training.cnn format
def pickleTrainingData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    train_data = np.empty((0, 32*32*3))
    train_labels = []

    for i in range(1, 2):
        fileNameDataBatch = './cifar-10-batches-py/data_batch_' + str(i)
        batch = unpickle(fileNameDataBatch)
        train_data = np.vstack((train_data, batch[b'data']))
        train_labels += batch[b'labels']

    train_labels = np.array(train_labels)
    train_data = train_data.reshape(-1, 32, 32, 3) / 255.0
    
    # !!!!! NOTICE HOW THE DATA IS SAVED !!!!!!
    # Will be returned in form of:
    # train_label, train_data  = getDataBack()
    pickle.dump([train_labels,train_data], open('./train.cnn', 'wb'))

pickleTrainingData()

In [4]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)

In [5]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
entry_point = "cnnScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "2.2",
    "py_version": "py37",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "role": role,
    "base_job_name": "cnn-model",
}

estimator = TensorFlow(**estimator_parameters)

In [6]:
inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: cnn-model-2024-11-09-15-58-07-123


2024-11-09 15:58:08 Starting - Starting the training job...
2024-11-09 15:58:23 Starting - Preparing the instances for training...
2024-11-09 15:58:44 Downloading - Downloading input data...
2024-11-09 15:59:29 Downloading - Downloading the training image..2024-11-09 15:59:52.724160: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2024-11-09 15:59:52.724843: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-11-09 15:59:52.758453: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2024-11-09 15:59:54,210 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-11-09 15:59:54,218 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-09 15:59:54,424 sagemaker-training-t

In [7]:
# CODE HELPER 2
def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])
    
    num_samples_to_select = 350
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]
    
    return selected_test_data, selected_test_labels

In [8]:
new_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='new-tensorflow-endpoint')

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: cnn-model-2024-11-09-16-00-54-639
INFO:sagemaker:Creating endpoint-config with name new-tensorflow-endpoint
INFO:sagemaker:Creating endpoint with name new-tensorflow-endpoint


----!

In [9]:
test_data, test_labels = getTestData()

result = new_predictor.predict(test_data)

In [10]:
from sklearn.metrics import accuracy_score
def getAccuracyOfPrediction(cnn_predictions, test_labels):
    cnn_predicted_labels = np.argmax(cnn_predictions, axis=1)
    accuracy = accuracy_score(test_labels, cnn_predicted_labels)
    print("Accuracy:", accuracy)
getAccuracyOfPrediction(result["predictions"], test_labels)

Accuracy: 0.46285714285714286


In [11]:
new_predictor.delete_endpoint(True)

INFO:sagemaker:Deleting endpoint configuration with name: new-tensorflow-endpoint
INFO:sagemaker:Deleting endpoint with name: new-tensorflow-endpoint
